In [1]:
from sympy import sqrt, I, series, limit, symbols, factorial, cos, sin, atan
from sympy.core.add import Add
from sympy.core.mul import Mul
from sympy.physics.secondquant import B, Bd, CreateBoson, AnnihilateBoson, Commutator, KroneckerDelta

In [2]:
# Eliminates all powers
def flatten(expr):
    expr = expr.expand()
    if expr.is_Add: return Add(*[flatten(arg) for arg in expr.args])
    elif expr.is_Mul: 
        factors = []
        for arg in expr.args:
            if arg.is_Pow and arg.exp.is_Integer and arg.exp > 0:
                factors.extend([arg.base] * int(arg.exp))
            else:
                factors.append(flatten(arg))
        return Mul(*factors, evaluate=False)
    elif expr.is_Pow and expr.exp.is_Integer and expr.exp > 0:
        return Mul(*[flatten(expr.base)] * int(expr.exp), evaluate=False)
    return expr

# Puts a polynomial expression of bosonic operators into the equivalent normal ordered form
def wick(expr):
    expr = flatten(expr)
    if expr.is_Add: return Add(*[wick(arg) for arg in expr.args])
    elif expr.is_Mul:
        commuting = [x for x in expr.args if not isinstance(x, AnnihilateBoson) and not isinstance(x, CreateBoson)]
        noncommuting = [x for x in expr.args if isinstance(x, AnnihilateBoson) or isinstance(x, CreateBoson)]
        n = len(noncommuting)
        if n == 0 or n == 1: return expr
        else:
            for i in range(n-1):
                c = Commutator(noncommuting[i], noncommuting[i+1])
                if isinstance(c, KroneckerDelta): c = 0
                if isinstance(noncommuting[i], AnnihilateBoson) and isinstance(noncommuting[i+1], CreateBoson):
                    expr1 = [c] + noncommuting[:i] + noncommuting[i+2:]
                    expr2 = noncommuting[:i] + [noncommuting[i+1], noncommuting[i]] + noncommuting[i+2:]
                    return wick(Mul(*commuting) * (wick(Mul(*expr1)) + wick(Mul(*expr2))))
            return expr
    else:
        return expr

# Computes the vacuum expectation value of a polynomial expression of bosonic operators
vev = lambda expr: Add(*[x for x in wick(expr).args if not any(isinstance(y, AnnihilateBoson) or isinstance(y, CreateBoson) for y in x.args)])

In [101]:
# Verifies the above functions work by using them to reproduce the main results of the paper:
# Quantum Theory of the Josephson Junction between Finite Islands

ng, x = symbols('n_g x', real = True)
N, EJ, EC = symbols("N E_J E_C", positive = True)
u_s, u_p, u_m, u_0 = symbols('u_s, u_p, u_m, u_0')
eps = sqrt(2*EC*EJ + EJ**2/N**2)
u_p = (EJ + N * eps) / sqrt(4 * N * eps * EJ)
u_m = (EJ - N * eps) / sqrt(4 * N * eps * EJ)
u_0 = ng * sqrt(2 * EC**2 * EJ / eps**3)
u_s = u_p + u_m

b = B('b')
bd = Bd('b')
a = u_p*b - u_m*bd + I*u_0*(u_p+u_m)
ad = u_p*bd - u_m*b - I*u_0*(u_p+u_m)
p = (a-ad)/(I*sqrt(2))
pp = p - ng/sqrt(N)

expr = -EC/2*(b*pp*ad*p*a*bd - pp*ad*p*a)
freq = eps + vev(expr)
assert((freq - series(freq, ng, 0, 3)).simplify()==0)
assert(freq.diff(ng).subs(ng,0)==0)
freq_0 = limit(freq.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
freq_2 = limit((N**2 * (freq.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng/N)**2
freq = (limit(((freq_0+freq_2)/sqrt(EC * EJ)).subs(EC, x * EJ), x, 0) * sqrt(EC * EJ)).simplify()
print('Transmon qubit frequency times Planck\'s constant:')
display(freq)

zeroth_order = vev(sqrt(N)*p)
first_order = vev(-ad*p*a/(4*sqrt(N)))
for k in range(1,5): first_order += EC*sqrt(N)/(2*eps*k*factorial(k)) * vev(p*bd**k) * (vev(b**k*pp*ad*p*a) + vev(b**k*ad*p*a*pp))
all_orders = (zeroth_order + first_order).simplify()
all_orders.diff(ng, 3).subs(ng, 0).simplify()
suscept = all_orders.diff(ng)
assert((suscept - series(suscept, ng, 0, 3)).simplify() == 0)
assert(suscept.diff(ng).subs(ng, 0) == 0)
suscept_0 = limit(suscept.subs(ng, 0).subs(N, x * EJ/EC), x, "oo")
suscept_2 = limit((N**4 * (suscept.diff(ng, 2)/2).subs(ng, 0)).subs(N, x * EJ/EC), x, "oo")*(ng**2)/(N**4)
suscept = suscept_0 + suscept_2
print('Transmon dimensionless charge susceptibility:')
display(suscept.simplify())

Transmon qubit frequency times Planck's constant:


sqrt(2)*sqrt(E_C)*sqrt(E_J)*(N**2 - n_g**2/4)/N**2

Transmon dimensionless charge susceptibility:


1 - 3*E_J*n_g**2/(4*E_C*N**4)

In [102]:
# define spin operators
S = symbols('S', positive = True)
th = symbols('theta', real = True)
a, ad = B('a'), Bd('a')
sz = S - ad*a
sp = sqrt(2*S)*(1-ad*a/(4*S))*a
sm = ad*sqrt(2*S)*(1-ad*a/(4*S))

sx = (sp+sm)/2
Sz = sz*cos(th)-sx*sin(th)
Sx = sx*cos(th)+sz*sin(th)

# define bosonic operators
hw = symbols(r'\hbar\omega', positive = True)
l = symbols(r'\lambda', real = True)
c, cd = B('c'), Bd('c')
b, bd = c - sqrt(2*S)*l*cos(th)/(4*hw), cd - sqrt(2*S)*l*cos(th)/(4*hw)

# define Hamiltonians
J, U = symbols(r'J U', positive = True)
gm = symbols(r'\gamma_-', real = True)
H_M = -J*Sx + U/(2*S)*Sz**2 - (gm/2)*Sz
H_L = hw*bd*b
H_LM = l/(2*sqrt(2*S))*Sz*(b+bd)
H = (H_M + H_L + H_LM).expand()

sqrtS = symbols(r'\sqrt{S}', positive = True)
H_ = []
max_order = 4
h = (H/S).subs(S, sqrtS**2)
for i in range(max_order+1):
    H_.append(((h.series(1/sqrtS, 0,i+1).removeO() - h.series(1/sqrtS, 0,i).removeO()).subs(sqrtS, sqrt(S))*S).expand())


In [103]:
display(H_[0])
display(H_[1])
display(H_[2])
display(H_[3])
display(H_[4])

-J*S*sin(theta) + S*U*cos(theta)**2/2 - S*\gamma_-*cos(theta)/2 - S*\lambda**2*cos(theta)**2/(8*\hbar\omega)

-sqrt(2)*J*sqrt(S)*cos(theta)*AnnihilateBoson(a)/2 - sqrt(2)*J*sqrt(S)*cos(theta)*CreateBoson(a)/2 - sqrt(2)*sqrt(S)*U*sin(theta)*cos(theta)*AnnihilateBoson(a)/2 - sqrt(2)*sqrt(S)*U*sin(theta)*cos(theta)*CreateBoson(a)/2 + sqrt(2)*sqrt(S)*\gamma_-*sin(theta)*AnnihilateBoson(a)/4 + sqrt(2)*sqrt(S)*\gamma_-*sin(theta)*CreateBoson(a)/4 + sqrt(2)*sqrt(S)*\lambda**2*sin(theta)*cos(theta)*AnnihilateBoson(a)/(8*\hbar\omega) + sqrt(2)*sqrt(S)*\lambda**2*sin(theta)*cos(theta)*CreateBoson(a)/(8*\hbar\omega)

J*sin(theta)*CreateBoson(a)*AnnihilateBoson(a) + U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)/4 + U*sin(theta)**2*AnnihilateBoson(a)**2/4 + U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)/4 + U*sin(theta)**2*CreateBoson(a)**2/4 - U*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a) + \gamma_-*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)/2 + \hbar\omega*CreateBoson(c)*AnnihilateBoson(c) - \lambda*sin(theta)*AnnihilateBoson(a)*AnnihilateBoson(c)/4 - \lambda*sin(theta)*AnnihilateBoson(a)*CreateBoson(c)/4 - \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(c)/4 - \lambda*sin(theta)*CreateBoson(a)*CreateBoson(c)/4 + \lambda**2*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a)/(4*\hbar\omega)

sqrt(2)*J*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(8*sqrt(S)) + sqrt(2)*J*cos(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(8*sqrt(S)) + sqrt(2)*U*sin(theta)*cos(theta)*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)/(4*sqrt(S)) + sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(a)/(4*sqrt(S)) + 3*sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(8*sqrt(S)) + 3*sqrt(2)*U*sin(theta)*cos(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(8*sqrt(S)) - sqrt(2)*\gamma_-*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(16*sqrt(S)) - sqrt(2)*\gamma_-*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)/(16*sqrt(S)) - sqrt(2)*\lambda*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*AnnihilateBoson(c)/(4*sqrt(S)) - sqrt(2)*\lambda*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(c)/(4*sqrt(S)) - sqrt(2)*\lambda**2*sin(theta)*cos(theta)*CreateBoson(a)*AnnihilateBoson(a)**2/(32*sqrt(S)*\hbar\omega) - sqrt(2)*\lambda**2*sin(theta)*cos(theta)*Cr

-U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)**2/(16*S) - U*sin(theta)**2*AnnihilateBoson(a)*CreateBoson(a)**2*AnnihilateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)*AnnihilateBoson(a)**3/(16*S) - U*sin(theta)**2*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(a)/(16*S) - U*sin(theta)**2*CreateBoson(a)**2*AnnihilateBoson(a)**2/(8*S) - U*sin(theta)**2*CreateBoson(a)**3*AnnihilateBoson(a)/(16*S) + U*cos(theta)**2*CreateBoson(a)*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)/(2*S) + \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2*AnnihilateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)*AnnihilateBoson(c)/(16*S) + \lambda*sin(theta)*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(c)/(16*S)

In [104]:
from sympy import solve
t = symbols('t', real = True)
t_ = solve((H_[0].diff(th).subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))*(1+t**2)**2*2*hw/S).simplify(), t)
freqs = [sqrt(J*(J*(1+t**2)**2/(4*t**2) + 2*U*t/(1+t**2))) for t in t_] 

In [7]:
# Verifies the analytical formula for frequency reproduces the numerical result from the paper:
# Quantum Theory of the Josephson Junction between Finite Islands

from scipy.linalg import eigh_tridiagonal
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from sympy import Integer, solve

def E(k, EC, EJ, ng, N):
    D = EC * (np.arange(-N, N+1)-ng)**2
    n = np.arange(-N, N)
    O = -(EJ/(2*N)) * np.sqrt(N*(N+1) - (n*(n+1)))
    eigs =  eigh_tridiagonal(D, O, eigvals_only = True, select = 'i', select_range = (k, k), tol = 1e-14)
    return eigs[0]

EC=1
EJ=10**3
N=10**6/2
ngs = np.array([int(ng) for ng in np.linspace(-N, N, 102)[1:-1]])
numerical = np.array([E(1, EC, EJ, ng, N) - E(0, EC, EJ, ng, N) for ng in tqdm(ngs)])

t = symbols('t', real = True)
t_ = solve((H_[0].diff(th).subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))*(1+t**2)**2*2*hw/S).simplify(), t)
freqs = [sqrt(J*(J*(1+t**2)**2/(4*t**2) + 2*U*t/(1+t**2))) for t in t_] 
EC = Integer(EC)
EJ = Integer(EJ)
N = Integer(N)
ngs = np.array([Integer(ng) for ng in ngs])
analytical = np.array([complex(freqs[3].subs({J: EJ/N, U: 2*N*EC, hw: 1, l: 0, gm: 4*ng*EC, S: N}).evalf()) for ng in tqdm(ngs)])

100%|██████████| 100/100 [00:42<00:00,  2.38it/s]


In [105]:

plt.plot(ngs/N, numerical / float(sqrt(2*EJ*EC)), label = 'numerical', color = 'black')
plt.plot(ngs/N, analytical / float(sqrt(2*EJ*EC)), label = 'analytical', linestyle = '--', color = 'red')
plt.xlabel(r'$n_\mathrm{g}/N$')
plt.ylabel(r'$\hbar \omega_\mathrm{q}~[\sqrt{2E_\mathrm{J}E_\mathrm{C}}$')
plt.title(r'$2N=10^6,\quad E_\mathrm{J}/E_\mathrm{C}=10^3$')
plt.legend()
plt.show()

TypeError: Cannot convert expression to float

In [106]:
e0, d, lt = symbols('epsilon_0, Delta, lambda_t', real = True)
Ep, Em = symbols('E_+, E_-', positive = True)
Np, Nm = symbols('N_+, N_-', positive = True)
Ep_sub, Em_sub = [sqrt((hw**2+e0**2-4*d**2)/2 + sign * sqrt(((hw**2-e0**2+4*d**2)/2)**2 + 4*lt**2*hw*(e0-2*d))) for sign in [1,-1]]
Np_sub, Nm_sub = [1/(sqrt(4*(e0-2*d)*E)*sqrt(1+4*hw*lt**2*(e0-2*d)/(hw**2-E**2)**2)) for E in [Ep, Em]]
qp, qpd, qm, qmd = B('qp'), Bd('qp'), B('qm'), Bd('qm')
a_sub = Np*((Ep+e0-2*d)*qp+(-Ep+e0-2*d)*qpd) + Nm*((Em+e0-2*d)*qm+(-Em+e0-2*d)*qmd)
ad_sub = Np*((Ep+e0-2*d)*qpd+(-Ep+e0-2*d)*qp) + Nm*((Em+e0-2*d)*qmd+(-Em+e0-2*d)*qm)
c_sub = Np*((2*lt*(e0-2*d))/(Ep-hw)*qp + (2*lt*(e0-2*d))/(-Ep-hw)*qpd) + Nm*((2*lt*(e0-2*d))/(Em-hw)*qm + (2*lt*(e0-2*d))/(-Em-hw)*qmd)
cd_sub = Np*((2*lt*(e0-2*d))/(Ep-hw)*qpd + (2*lt*(e0-2*d))/(-Ep-hw)*qp) + Nm*((2*lt*(e0-2*d))/(Em-hw)*qmd + (2*lt*(e0-2*d))/(-Em-hw)*qm)

# define quadratic Hamiltonian
H_2 = wick((e0*(ad*a) + d*(ad*ad+a*a)+lt*(a+ad)*(c+cd)+hw*cd*c+d).subs({a: a_sub, ad: ad_sub, c: c_sub, cd: cd_sub}))

# verify Bogoliubov diagonalization and commutation relations
extract = lambda t1, t2: Add(*[x for x in H_2.args if x.has(t1) and x.has(t2)]).subs({t1:1, t2:1})
for t1, t2 in [(qp, qm), (qpd, qm), (qp, qmd), (qpd, qmd)]: 
    print(0 == (extract(t1, t2)*(Ep+hw)*(Ep-hw)*(Em+hw)*(Em-hw)/Np/Nm).subs({Ep:Ep_sub, Em:Em_sub}).simplify())
for i in range(2):
    print(Ep == extract(qp, qpd).subs({Np:Np_sub, lt:solve(Ep-Ep_sub, lt)[i]}).simplify())
    print(Em == extract(qm, qmd).subs({Nm:Nm_sub, lt:solve(Em-Em_sub, lt)[i]}).simplify())
commute = lambda t1, t2: Add(*[x for x in Commutator(t1, t2).args if not x.has(KroneckerDelta)])
for t1, t2 in [(a_sub, ad_sub), (c_sub, cd_sub)]: 
    print(1 == commute(t1, t2).subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).simplify())
for t1, t2 in [(a_sub, c_sub), (a_sub, cd_sub)]:
    print(0 == commute(t1, t2).subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).simplify())
print('Verification complete.')

H_2 = extract(qpd, qp)*qpd*qp + extract(qmd, qm)*qmd*qm

e0_sub = ((gm/2)*cos(th)+J*sin(th)+(U/2)*sin(th)**2 - (U-l**2/(4*hw))*cos(th)**2)
d_sub = (U/4)*sin(th)**2
lt_sub = (-l/4)*sin(th)
#H_2 = H_2.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub})

True
True
True
True
True
True
True
True
True
True
True
True
Verification complete.


In [107]:
for i in range(5):
    display(H_[i].subs(gm,0).subs(cos(th),0).subs(sin(th),1).simplify())

w = wick(H_[4].subs({a:a_sub, ad:ad_sub, c:c_sub, cd:cd_sub}))

-J*S

0

(4*J*CreateBoson(a)*AnnihilateBoson(a) + U*AnnihilateBoson(a)*CreateBoson(a) + U*AnnihilateBoson(a)**2 + U*CreateBoson(a)*AnnihilateBoson(a) + U*CreateBoson(a)**2 + 4*\hbar\omega*CreateBoson(c)*AnnihilateBoson(c) - \lambda*AnnihilateBoson(a)*AnnihilateBoson(c) - \lambda*AnnihilateBoson(a)*CreateBoson(c) - \lambda*CreateBoson(a)*AnnihilateBoson(c) - \lambda*CreateBoson(a)*CreateBoson(c))/4

0

(-U*AnnihilateBoson(a)*CreateBoson(a)*AnnihilateBoson(a)**2 - U*AnnihilateBoson(a)*CreateBoson(a)**2*AnnihilateBoson(a) - U*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(a) - U*CreateBoson(a)*AnnihilateBoson(a)**3 - U*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(a) - 2*U*CreateBoson(a)**2*AnnihilateBoson(a)**2 - U*CreateBoson(a)**3*AnnihilateBoson(a) + \lambda*CreateBoson(a)*AnnihilateBoson(a)**2*AnnihilateBoson(c) + \lambda*CreateBoson(a)*AnnihilateBoson(a)**2*CreateBoson(c) + \lambda*CreateBoson(a)**2*AnnihilateBoson(a)*AnnihilateBoson(c) + \lambda*CreateBoson(a)**2*AnnihilateBoson(a)*CreateBoson(c))/(16*S)

KeyboardInterrupt: 

In [85]:
w_ppmm = Add(*[term for term in w.args if term.count(qpd)==1 and term.count(qp)==1 and term.count(qmd)==1 and term.count(qm)==1]).subs({qp:1, qpd:1, qm:1, qmd:1})

In [139]:
gp = l**2/hw
Ut = U - gp/4
vevp = (vev(Sz.subs({a:a_sub, ad:ad_sub, c:c_sub, cd:cd_sub})*qpd)/sqrt(S)).limit(S, 'oo')*sqrt(S)
vevp
#vevm = ((vev(Sz.subs({a:a_sub, ad:ad_sub, c:c_sub, cd:cd_sub})*qmd/sqrt(S))).expand().limit(S, 'oo')*sqrt(S)).simplify()
#vevp = vevp.subs(Np, Np_sub).subs(Nm, Nm_sub).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub})
#display(vevp.subs({cos(th):sqrt(1-(J/Ut)**2), sin(th):-J/Ut}).subs(gm, 0).limit(hw, 0))
#display(vevm.subs({cos(th):sqrt(1-(J/Ut)**2), sin(th):-J/Ut}).subs(gm, 0).limit(hw, 0))


sqrt(2)*N_+*sqrt(S)*(2*Delta - epsilon_0)*sin(theta)

In [98]:
Ep

E_+

In [83]:
w.subs(e0, e0_sub).subs(d, d_sub).subs(gm, 0).subs({cos(th):0, sin(th):1}).simplify()
w_pppp = Add(*[term for term in w.args if term.count(qpd)==2 and term.count(qp)==2]).subs({qp:1, qpd:1})
w_mmmm = Add(*[term for term in w.args if term.count(qmd)==2 and term.count(qm)==2]).subs({qm:1, qmd:1})
w_ppmm = Add(*[term for term in w.args if term.count(qpd)==1 and term.count(qp)==1 and term.count(qmd)==1 and term.count(qm)==1]).subs({qp:1, qpd:1, qm:1, qmd:1})
w_ppmm_reduced = w_ppmm.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0:e0_sub, d:d_sub, lt:lt_sub}).subs(gm, 0).subs({cos(th):0, sin(th):1}).simplify()

KeyboardInterrupt: 

In [77]:
w_ppmm_reduced = w_ppmm_reduced.subs({hw:1, U:1, J:1}).simplify()
limit(w_ppmm_reduced, l, 0)
#from sympy import pi
#c_sub_reduced = c_sub.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub}).subs(th, pi/2)
#zero_coupling 

0

In [ ]:
from sympy import solve

Ep_0 = Ep_sub.subs({e0: e0_sub, d: d_sub, lt: lt_sub}).subs(l,0)
Em_0 = Em_sub.subs({e0: e0_sub, d: d_sub, lt: lt_sub}).subs(l,0)
t = symbols('t', real = True)
t_ = solve((H_[0].diff(th).subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))*(1+t**2)**2).simplify(), t)
display(Ep_0.simplify())
display(Em_0.simplify())
Ep_0 = Ep_0.subs({cos(th):(1-t**2)/(1+t**2), sin(th):2*t/(1+t**2)})
Em_0 = Em_0.subs({cos(th):(1-t**2)/(1+t**2), sin(th):2*t/(1+t**2)})
Ep_0_ = [Ep_0.subs(t, ti) for ti in t_]
Em_0_ = [Em_0.subs(t, ti) for ti in t_]

In [ ]:
from sympy import Integer
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
EC=1
EJ=10**3
N=10**6/2
ngs = np.array([int(ng) for ng in np.linspace(-N, N, 12)[1:-1]])
#numerical = np.array([E(1, EC, EJ, ng, N) - E(0, EC, EJ, ng, N) for ng in tqdm(ngs)])
EC = Integer(EC)
EJ = Integer(EJ)
N = Integer(N)

ngs = np.array([Integer(ng) for ng in ngs])
for i in range(len(Ep_0_)):
    Ep_0_data = np.array([complex(Ep_0_[i].subs({J: EJ/N, U: 2*N*EC, hw: 1, l: 0, gm: 4*ng*EC, S: N}).evalf()).real for ng in tqdm(ngs)])
    print(Ep_0_data)
    plt.scatter(ngs, Ep_0_data)
    plt.title(f'E_+ branch {i}')
    plt.show()
    Em_0_data = np.array([complex(Em_0_[i].subs({J: EJ/N, U: 2*N*EC, hw: 1, l: 0, gm: 4*ng*EC, S: N}).evalf()).real for ng in tqdm(ngs)])
    plt.scatter(ngs, Em_0_data)
    plt.title(f'E_- branch {i}')
    plt.show()
#analytical = np.array([complex(freqs[3].subs({J: EJ/N, U: 2*N*EC, hw: 1, l: 0, gm: 4*ng*EC, S: N}).evalf()) for ng in tqdm(ngs)])

In [ ]:
from sympy import pi

Szt_approx = S-ad_sub*a_sub
Sxt_approx = (a_sub+ad_sub)/sqrt(2)
Sz_approx = Szt_approx*cos(th)-Sxt_approx*sin(th)
Szp_approx = vev(Sz_approx*qpd).simplify()
Szm_approx = vev(Sz_approx*qmd).simplify()
Szp_approx = Szp_approx.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub}).subs(sin(th),1).subs(cos(th),0)
Szm_approx = Szm_approx.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub}).subs(sin(th),1).subs(cos(th),0)

#Szp_approx.simplify()
#display(Szp_approx.subs(th, pi/2).subs(l, 0).simplify())
#Szm_approx = Szm_approx.subs({Np:Np_sub, Nm:Nm_sub}).subs({Ep:Ep_sub, Em:Em_sub}).subs({e0: e0_sub, d: d_sub, lt: lt_sub})
#display(Szm_approx.subs(th, pi/2).subs(l, 0).simplify())
#x = (series(Szp_approx, l, 0, 3).removeO() - series(Szp_approx, l, 0, 2).removeO()).simplify()

#Szp_approx.subs(sin(th), 2*t/(1+t**2)).subs(cos(th), (1-t**2)/(1+t**2))


In [ ]:
Szp_approx

In [ ]:
limit(Szm_approx, l, 0).simplify()

In [ ]:
x=H_[4].subs({a:a_sub, ad:ad_sub, c:c_sub, cd:cd_sub})
w=wick(x)

In [ ]:
p4_m0_combos = [qpd*qpd*qp*qp]
p3_m1_combos = [qpd*qpd*qp*qm, qpd*qpd*qm*qp, qpd*qmd*qp*qp, qmd*qpd*qp*qp]
p2_m2_combos = [qpd*qpd*qm*qm, qmd*qmd*qp*qp, qpd*qmd*qp*qm, qpd*qmd*qm*qp, qmd*qpd*qp*qm, qmd*qpd*qm*qp]
p1_m3_combos = [qpd*qmd*qm*qm,qmd*qpd*qm*qm, qmd*qmd*qp*qm, qmd*qmd*qm*qp]
p0_m4_combos = [qmd*qmd*qm*qm]
combos = p4_m0_combos + p3_m1_combos + p2_m2_combos + p1_m3_combos + p0_m4_combos
reduced = Add(*[x for x in w.args if any(combo in x.args for combo in combos)])
len(reduced)

In [ ]:
x.subs(l, 0).expand()

In [ ]:
display(H_[3])

In [82]:
e0_ = solve((hw**2-e0**2+4*d**2)**2/4 + 4*lt**2*hw*(e0-2*d), e0)
for e0 in tqdm(e0_): display(e0.simplify())

KeyboardInterrupt: 

In [16]:
epsilon_p, epsilon_m, alpha_p, alpha_m, beta, gamma = symbols('epsilon_+, epsilon_-, alpha_+, alpha_-, beta, gamma', real = True)

b_l, b_r = B('L'), B('R')
bd_l, bd_r = Bd('L'), Bd('R')
b_p, b_m = (b_l + b_r)/sqrt(2), (b_l - b_r)/sqrt(2)
bd_p, bd_m = (bd_l + bd_r)/sqrt(2), (bd_l - bd_r)/sqrt(2)
#b_p, b_m = B('p'), B('m')
#bd_p, bd_m = Bd('p'), Bd('m')

H = epsilon_p*bd_p*b_p + epsilon_m*bd_m*b_m + alpha_p*bd_p*bd_p*b_p*b_p + alpha_m*bd_m*bd_m*b_m*b_m + 2*(beta+gamma)*bd_p*b_p*bd_m*b_m + gamma*(bd_p*bd_p*b_m*b_m + bd_m*bd_m*b_p*b_p)

In [37]:
display(wick(bd_p*b_p))
display(wick(bd_m*b_m))
display(wick(bd_p*bd_p*b_p*b_p))
display(wick(bd_m*bd_m*b_m*b_m))
display(wick(bd_p*bd_m*b_p*b_m))
display(wick(bd_p*bd_p*b_m*b_m + bd_m*bd_m*b_p*b_p))

CreateBoson(L)*AnnihilateBoson(L)/2 + CreateBoson(L)*AnnihilateBoson(R)/2 + CreateBoson(R)*AnnihilateBoson(L)/2 + CreateBoson(R)*AnnihilateBoson(R)/2

CreateBoson(L)*AnnihilateBoson(L)/2 - CreateBoson(L)*AnnihilateBoson(R)/2 - CreateBoson(R)*AnnihilateBoson(L)/2 + CreateBoson(R)*AnnihilateBoson(R)/2

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 - CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 - CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 - CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 - CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 - CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 - CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/2 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/2 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/2 - CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/2 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/2 - CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/2 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/2 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/2

In [41]:
# BEGIN: Extract unequal ladder operators
w = wick(bd_p*bd_p*b_p*b_p)
unequal_terms = Add(*[term for term in w.args if term.count(b_l) + term.count(bd_l) == 1 or term.count(b_r) + term.count(bd_r) == 1])
# END: Extract unequal ladder operators
display(unequal_terms)

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson

CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(L)*CreateBoson(R)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(L)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(L)*AnnihilateBoson(R)*AnnihilateBoson(R)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson(L)/4 + CreateBoson(R)*CreateBoson(R)*AnnihilateBoson(L)*AnnihilateBoson